In [12]:
import json
import os
import time
from copy import deepcopy
from pathlib import Path

import numpy as np
import openai
import requests
import yaml
from loguru import logger as eval_logger
from openai import OpenAI


In [13]:
from datasets import load_dataset

task_name = 'AgriExam'
folder = '/workdir/important_datasets/AGRIVQA/'
dataset_name = "parquet"
data_files = {
    "dev": folder+task_name+"/dev-00000-of-00001.parquet",
    "test": folder+task_name+"/test-00000-of-00001.parquet",
    "validation": folder+task_name+"/validation-00000-of-00001.parquet"
}
split = "dev"

dataset = load_dataset(dataset_name,data_files=data_files, split=split)

#### Here, the .yaml file is read and converted into a dictionary named config.

In [14]:
__file__= "/workdir/agrivqa_tasks/CABBAGE_agriexam/CABBAGE_agriexam.yaml"

#rule_dict = json.load(open(os.path.join(os.path.dirname(os.path.abspath(__file__)), "rule.json"), "r"))

with open(Path(__file__).parent / "CABBAGE_agriexam.yaml", "r") as f:
    raw_data = f.readlines()
    safe_data = []
    for i, line in enumerate(raw_data):
        # remove function definition since yaml load cannot handle it
        if "!function" not in line:
            safe_data.append(line)

    config = yaml.safe_load("".join(safe_data))

#### Later


#### Here, as with all other tasks, the visuals and text are prepared for the models.

In [15]:
def parse_options(options):
    option_letters = [chr(ord("A") + i) for i in range(len(options))]
    choices_str = "\n".join([f"{option_letter}. {option}" for option_letter, option in zip(option_letters, options)])
    return choices_str

def CABBAGE_AgriExam_doc_to_text(doc, lmms_eval_specific_kwargs=None):
    question = doc['question']
    if lmms_eval_specific_kwargs is None:
        lmms_eval_specific_kwargs = {}
    pre_prompt = lmms_eval_specific_kwargs.get("pre_prompt", "")
    post_prompt = lmms_eval_specific_kwargs.get("post_prompt", "")
    if doc['options']:
        options = parse_options(doc['options'])
        return f"{pre_prompt}\nQuestion: {question}\n\nOptions:\n{options}\n{post_prompt}"
    return f"{pre_prompt}\n\nQuestion: {question}{post_prompt}"

In [16]:
doc = dataset[3]
print(CABBAGE_AgriExam_doc_to_text(doc,config['lmms_eval_specific_kwargs']['default']))


Question: Study of grasses?

Options:
A. Agroforestry
B. Agronomy
C. Zoology
D. Agrostology
E. All of above

Answer with the option's letter from the given choices directly.


In [41]:
def CABBAGE_AgriExam_process_results(doc, results):
    # I know this is weird, but it's how llava parse it.
    target = doc['answer'].strip().lower()
    pred = results[0].strip().lower()
    if pred == target:
        return {"exact_match": 1.0}
    # pattern: ^[A-Z]\. .*
    if len(pred) >= 2 and pred[1] == ".":
        result = 1.0 if pred[0] == target else 0.0
        return {"exact_match": result}
    return {"exact_match": 0.0}


In [34]:
doc

{'id': 'dev_AgriExam_4',
 'question': 'Study of grasses?',
 'options': ['Agroforestry',
  'Agronomy',
  'Zoology',
  'Agrostology',
  'All of above'],
 'answer': 'D',
 'category': 'Plant Taxonomy and Scientific Naming',
 'question_type': 'multiple-choice',
 'metadata': {'language': 'English',
  'license': '',
  'source': 'AgriExam',
  'url': 'https://www.agriexam.com/agriculture-field-officer-2015',
  'verbose_answer': 'Agrostology'}}

In [37]:
pred='D.'

In [ ]:
pred.

True

In [42]:
CABBAGE_AgriExam_process_results(doc, ['D.'])

{'exact_match': 1.0}